# Setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import  TfidfVectorizer
from scipy.sparse import hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

In [2]:
df = pd.read_csv("csv\\raw_data_labeled_ready.csv")
df = df[df["y"].notnull()]
df.head()

,id,title,formats,thumbnails,description,upload_date,uploader,uploader_id,uploader_url,channel_id,...,tbr,protocol,preference,http_headers,release_date,release_year,days_since_upload,thumbnails_links,y,views_per_day
0,JyGGMyR3x5I,Machine Learning: Tutorial prático usando apen...,"[{'asr': 48000, 'filesize': 16323674, 'format_...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",✅ALURA COM 10% DE DESCONTO: https://www.alura....,2019-11-08,Filipe Deschamps,UCU5JicSrEM5A63jkJ2QvGYw,http://www.youtube.com/channel/UCU5JicSrEM5A63...,UCU5JicSrEM5A63jkJ2QvGYw,...,NaN,NaN,NaN,NaN,NaN,NaN,825.0,https://i.ytimg.com/vi_webp/JyGGMyR3x5I/maxres...,0.0,219.296970
1,0PrOA2JK6GQ,Machine Learning Explicado,"[{'asr': 48000, 'filesize': 5203175, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Machine learning (ou aprendizado de máquina) é...,2021-02-25,Ciência Todo Dia,CienciaTodoDia,http://www.youtube.com/user/CienciaTodoDia,UCn9Erjy00mpnWeLnRqhsA1g,...,NaN,NaN,NaN,NaN,NaN,NaN,350.0,https://i.ytimg.com/vi/0PrOA2JK6GQ/maxresdefau...,0.0,556.602857
2,t5z5lyrb-7s,Introdução ao Machine Learning (ML de Zero a 1...,"[{'asr': 48000, 'filesize': 2603732, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",This video is also available in English: https...,2019-11-13,TensorFlow,UC0rqucBdTuFTjJiefW5t-IQ,http://www.youtube.com/channel/UC0rqucBdTuFTjJ...,UC0rqucBdTuFTjJiefW5t-IQ,...,NaN,NaN,NaN,NaN,NaN,NaN,820.0,https://i.ytimg.com/vi_webp/t5z5lyrb-7s/maxres...,0.0,17.907317
3,5q87K1WaoFI,Computer Scientist Explains Machine Learning i...,"[{'asr': 48000, 'filesize': 9863039, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",WIRED has challenged computer scientist and Hi...,2021-08-18,WIRED,wired,http://www.youtube.com/user/wired,UCftwRNsjfRo08xYE31tkiyw,...,NaN,NaN,NaN,NaN,NaN,NaN,176.0,https://i.ytimg.com/vi/5q87K1WaoFI/maxresdefau...,0.0,5726.215909
4,qDmojpkd8rE,O que é Machine Learning (Aprendizado de Máqui...,"[{'format_id': '139', 'manifest_url': 'https:/...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Nesse vídeo vamos aprender o que é machine lea...,2019-06-21,Didática Tech,UC0BiVs5EYh57gzGVvhddjsA,http://www.youtube.com/channel/UC0BiVs5EYh57gz...,UC0BiVs5EYh57gzGVvhddjsA,...,NaN,NaN,NaN,NaN,NaN,NaN,965.0,https://i.ytimg.com/vi_webp/qDmojpkd8rE/maxres...,0.0,28.239378


In [2]:
# features = ["views_per_day", "days_since_upload", "duration", "view_count", "like_count"]
features = ["views_per_day", "days_since_upload", "duration", "view_count"]
# df[features].isnull().sum()

# Train test split

In [4]:
date = "2020-10-05"
Xtrain, Xval = df[df["upload_date"] < date].drop("y", axis=1), df[df["upload_date"] >= date].drop("y", axis=1)
ytrain, yval = df[df["upload_date"] < date]["y"], df[df["upload_date"] >= date]["y"]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((263, 74), (255, 74), (263,), (255,))

# Title modeling

In [5]:
title_train = Xtrain["title"]
title_val = Xval["title"]

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [6]:
Xtrain_wtitle = hstack([Xtrain[features], title_bow_train])
Xval_wtitle = hstack([Xval[features], title_bow_val])

In [7]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((263, 225), (255, 225))

In [8]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [9]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [10]:
average_precision_score(yval, p)

0.23206104430364005

In [11]:
roc_auc_score(yval, p)

0.5519785378940308

# active learning

70 samples that the model have more difficult to predict and 30 random choice samples

In [7]:
df_unlabeled = pd.read_csv("csv\\raw_data_labeled_ready.csv")
df_unlabeled = df_unlabeled[df_unlabeled["y"].isnull()].dropna(how="all")
df_unlabeled.shape

ParserError: Error tokenizing data. C error: out of memory

In [13]:
title_u = df_unlabeled["title"]
title_bow_u = title_vec.transform(title_u)

In [14]:
Xu_wtitle = hstack([df_unlabeled[features], title_bow_u])

In [15]:
pu = mdl.predict_proba(Xu_wtitle)[:, 1]

In [16]:
df_unlabeled['p'] = pu

In [17]:
mask_u = (df_unlabeled["p"] >= 0.327) & (df_unlabeled["p"] <= 1.)
mask_u.sum()

71

In [18]:
df_unlabeled[mask_u]

,id,title,formats,thumbnails,description,upload_date,uploader,uploader_id,uploader_url,channel_id,...,protocol,preference,http_headers,release_date,release_year,days_since_upload,thumbnails_links,y,views_per_day,p
463,R0vC31OXt-g,"Build, Train and Deploy Machine Learning Model...","[{'asr': 48000, 'filesize': 13799617, 'format_...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...","To learn more, please visit: https://aws.amazo...",2018-09-20,AWS Online Tech Talks,AWSwebinars,http://www.youtube.com/user/AWSwebinars,UCT-nPlVzJI-ccQXlxjSvJmw,...,NaN,NaN,NaN,NaN,NaN,1239.0,https://i.ytimg.com/vi_webp/R0vC31OXt-g/maxres...,NaN,124.841001,0.342
472,lVP7ywRq8p4,Azure Machine Learning Pipelines,"[{'asr': 48000, 'filesize': 4057061, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",This video talks about Azure Machine Learning ...,2019-11-07,Microsoft Developer,UCsMica-v34Irf9KVTh6xx-g,http://www.youtube.com/channel/UCsMica-v34Irf9...,UCsMica-v34Irf9KVTh6xx-g,...,NaN,NaN,NaN,NaN,NaN,826.0,https://i.ytimg.com/vi_webp/lVP7ywRq8p4/maxres...,NaN,10.430993,0.391
477,WZ7vS10KPAw,Model deployment and inferencing with Azure Ma...,"[{'asr': 48000, 'filesize': 5723810, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...","In this video, learn about the various deploym...",2021-07-23,Microsoft Azure,windowsazure,http://www.youtube.com/user/windowsazure,UC0m-80FnNY2Qb7obvTL_2fA,...,NaN,NaN,NaN,NaN,NaN,202.0,https://i.ytimg.com/vi_webp/WZ7vS10KPAw/maxres...,NaN,29.153465,0.383
479,jGwO_UgTS7I,Lecture 1 - Stanford CS229: Machine Learning -...,"[{'asr': 48000, 'filesize': 27046391, 'format_...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",For more information about Stanford’s Artifici...,2020-04-17,stanfordonline,stanfordonline,http://www.youtube.com/user/stanfordonline,UCBa5G_ESCn8Yd4vw5U-gIcg,...,NaN,NaN,NaN,NaN,NaN,664.0,https://i.ytimg.com/vi_webp/jGwO_UgTS7I/maxres...,NaN,1663.703313,0.476
507,x6f5JOPhci0,"How optimization for machine learning works, p...","[{'asr': 48000, 'filesize': 3892938, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Part of the End-to-End Machine Learning School...,2018-10-14,Brandon Rohrer,BrandonRohrer,http://www.youtube.com/user/BrandonRohrer,UCsBKTrp45lTfHa_p49I2AEQ,...,NaN,NaN,NaN,NaN,NaN,1215.0,https://i.ytimg.com/vi_webp/x6f5JOPhci0/maxres...,NaN,23.609053,0.451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625,gjOiHCh9CzQ,Jamie Oliver Chooses A Dish For Jamie's Italia...,"[{'asr': 48000, 'filesize': 3232706, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Jamie Oliver visits the MasterChef Canada kitc...,2018-07-25,MasterChef World,UC_znAZirr11Q5bPrdhKVLZQ,http://www.youtube.com/channel/UC_znAZirr11Q5b...,UC_znAZirr11Q5bPrdhKVLZQ,...,NaN,NaN,NaN,NaN,NaN,1296.0,https://i.ytimg.com/vi_webp/gjOiHCh9CzQ/maxres...,NaN,2677.839506,0.345
3635,8ak3lUxy_yU,How to Make CREAMY MUSHROOM RISOTTO Like an It...,"[{'asr': 48000, 'filesize': 6367863, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",My mushroom risotto recipe is a no-fail way to...,2021-05-30,Vincenzo's Plate,vincenzosplate,http://www.youtube.com/user/vincenzosplate,UCcsSowAamCLJv-xeF9geXoA,...,NaN,NaN,NaN,NaN,NaN,256.0,https://i.ytimg.com/vi_webp/8ak3lUxy_yU/maxres...,NaN,1720.156250,0.350
3648,Nlsm0RlC8zI,2006 WORLD CUP FINAL: Italy 1-1 France,"[{'asr': 48000, 'filesize': 809985, 'format_id...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",All the highlights from the famous Final at th...,2014-07-09,FIFA,FIFATV,http://www.youtube.com/user/FIFATV,UCpcTrCXblq78GZrTUTLWeBw,...,NaN,NaN,NaN,NaN,NaN,2773.0,https://i.ytimg.com/vi_webp/Nlsm0RlC8zI/maxres...,NaN,1877.732420,0.341
3652,iLZR7tihHbk,Elena feat. Glance - Mamma mia (He's italiano)...,"[{'asr': 48000, 'filesize': 1416086, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Subscribe to Cat Music Channel: https://goo.gl...,2014-05-21,Cat Music,catmusicoffice,

In [19]:
hard = df_unlabeled[mask_u]

In [20]:
random = df_unlabeled[~mask_u].sample(30, random_state=42)

In [21]:
pd.concat([hard, random]).to_csv("csv//active_label.csv", index=False)

In [95]:
!label-studio

^C


# Results of active learning

In [22]:
df_active = pd.read_csv("csv//active_label_labeled.csv")
df_active.head()

,Unnamed: 0,id,title,formats,thumbnails,description,upload_date,uploader,uploader_id,uploader_url,...,thumbnails_links,y,views_per_day,p,intent,annotator,annotation_id,created_at,updated_at,lead_time
0,597,3958,AWS Innovate - AI / ML Edition 2022 | How to u...,"[{'asr': 48000, 'filesize': 1903954, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...","Join Telegram [For Free Udemy Courses, Interns...",2022-02-03,Course Intern,UCZaxdoEXp3YvRSO9bcO7S9w,http://www.youtube.com/channel/UCZaxdoEXp3YvRS...,...,https://i.ytimg.com/vi_webp/DnHCPR8GDEE/maxres...,NaN,43.428571,0.353,É interessante,1,780,2022-02-09T23:56:34.601106Z,2022-02-09T23:56:41.951201Z,114.567
1,611,4056,Designing Safe and Reliable Autonomous Systems...,"[{'format_id': '139', 'manifest_url': 'https:/...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...","Chuchu Fan, Wilson Assistant Professor of Aero...",2022-02-04,MIT Corporate Relations,UCHWIPmNUwv8PLBpTyOyyP0Q,http://www.youtube.com/channel/UCHWIPmNUwv8PLB...,...,https://i.ytimg.com/vi_webp/seBrZhrXeRw/maxres...,NaN,2.166667,0.190,Não é interessante,1,878,2022-02-10T00:05:12.115902Z,2022-02-10T00:05:12.115902Z,3.139
2,3474,4055,Business Is Booming for the Italian Mafia Duri...,"[{'asr': 48000, 'filesize': 4420653, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",The coronavirus hit Italy like a freight train...,2020-08-27,VICE News,vicenews,http://www.youtube.com/user/vicenews,...,https://i.ytimg.com/vi_webp/IsNN6d_5oJ8/maxres...,NaN,7212.731203,0.305,Não é interessante,1,877,2022-02-10T00:05:08.466803Z,2022-02-10T00:05:08.466803Z,2.734
3,1814,4054,Facebook Stock Plummets 26% in ONE DAY - Lesso...,"[{'asr': 48000, 'filesize': 4154244, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",What can we learn from Facebook stocks big mar...,2022-02-03,Dave Lee on Investing,UCi2G5aQTN0YwGIfnt1u7Nbg,http://www.youtube.com/channel/UCi2G5aQTN0YwGI...,...,https://i.ytimg.com/vi_webp/U0SCKapxxXc/maxres...,NaN,11469.571429,0.232,Não é interessante,1,876,2022-02-10T00:05:05.223973Z,2022-02-10T00:05:05.223973Z,2.218
4,3020,4053,MASSIVE! Top 4 EXPLOSIVE ASIAN NFT Projects To...,"[{'asr': 48000, 'filesize': 5747012, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",✅ FOLLOW ME ON TWITTER 👉https://twitter.com/Do...,2022-02-03,Donte Walker NFT,UClzzOYzkXuXo2oo1XtsC1JA,http://www.youtube.com/channel/UClzzOYzkXuXo2o...,...,https://i.ytimg.com/vi/_kaU_a4x5Lg/hqdefault.j...,NaN,473.857143,0.133,Não é interessante,1,875,2022-02-10T00:05:02.543396Z,2022-02-10T00:05:02.543396Z,1.498


In [23]:
def y(x):
    if x == "É interessante":
        return 1
    if x == "Não é interessante":
        return 0
    return np.nan
df_active["y"] = df_active["intent"].apply(lambda x: y(x))
df_active = df_active.drop(["Unnamed: 0", "created_at", "updated_at", "lead_time", "annotation_id", "annotator", "intent"], axis=1)
df_active.head()

,id,title,formats,thumbnails,description,upload_date,uploader,uploader_id,uploader_url,channel_id,...,protocol,preference,http_headers,release_date,release_year,days_since_upload,thumbnails_links,y,views_per_day,p
0,3958,AWS Innovate - AI / ML Edition 2022 | How to u...,"[{'asr': 48000, 'filesize': 1903954, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...","Join Telegram [For Free Udemy Courses, Interns...",2022-02-03,Course Intern,UCZaxdoEXp3YvRSO9bcO7S9w,http://www.youtube.com/channel/UCZaxdoEXp3YvRS...,UCZaxdoEXp3YvRSO9bcO7S9w,...,NaN,NaN,NaN,NaN,NaN,7.0,https://i.ytimg.com/vi_webp/DnHCPR8GDEE/maxres...,1,43.428571,0.353
1,4056,Designing Safe and Reliable Autonomous Systems...,"[{'format_id': '139', 'manifest_url': 'https:/...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...","Chuchu Fan, Wilson Assistant Professor of Aero...",2022-02-04,MIT Corporate Relations,UCHWIPmNUwv8PLBpTyOyyP0Q,http://www.youtube.com/channel/UCHWIPmNUwv8PLB...,UCHWIPmNUwv8PLBpTyOyyP0Q,...,NaN,NaN,NaN,NaN,NaN,6.0,https://i.ytimg.com/vi_webp/seBrZhrXeRw/maxres...,0,2.166667,0.190
2,4055,Business Is Booming for the Italian Mafia Duri...,"[{'asr': 48000, 'filesize': 4420653, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",The coronavirus hit Italy like a freight train...,2020-08-27,VICE News,vicenews,http://www.youtube.com/user/vicenews,UCZaT_X_mc0BI-djXOlfhqWQ,...,NaN,NaN,NaN,NaN,NaN,532.0,https://i.ytimg.com/vi_webp/IsNN6d_5oJ8/maxres...,0,7212.731203,0.305
3,4054,Facebook Stock Plummets 26% in ONE DAY - Lesso...,"[{'asr': 48000, 'filesize': 4154244, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",What can we learn from Facebook stocks big mar...,2022-02-03,Dave Lee on Investing,UCi2G5aQTN0YwGIfnt1u7Nbg,http://www.youtube.com/channel/UCi2G5aQTN0YwGI...,UCi2G5aQTN0YwGIfnt1u7Nbg,...,NaN,NaN,NaN,NaN,NaN,7.0,https://i.ytimg.com/vi_webp/U0SCKapxxXc/maxres...,0,11469.571429,0.232
4,4053,MASSIVE! Top 4 EXPLOSIVE ASIAN NFT Projects To...,"[{'asr': 48000, 'filesize': 5747012, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",✅ FOLLOW ME ON TWITTER 👉https://twitter.com/Do...,2022-02-03,Donte Walker NFT,UClzzOYzkXuXo2oo1XtsC1JA,http://www.youtube.com/channel/UClzzOYzkXuXo2o...,UClzzOYzkXuXo2oo1XtsC1JA,...,NaN,NaN,NaN,NaN,NaN,7.0,https://i.ytimg.com/vi/_kaU_a4x5Lg/hqdefault.j...,0,473.857143,0.133


In [24]:
def mapper_y(x):
    y = df_unlabeled.loc[df_unlabeled["thumbnails_links"] == x, "id"].tolist()
    if len(y) != 0:
        return y[0]
    return np.nan

df_active["id"] = df_active["thumbnails_links"].apply(lambda x: mapper_y(x))

In [25]:
average_precision_score(df_active["y"], df_active["p"]), roc_auc_score(df_active["y"], df_active["p"])

(0.262961106290285, 0.46874999999999994)

In [26]:
df_active["new"] = 1

df_total = pd.concat([df, df_active.drop("p", axis=1)])

In [27]:
df_total["new"] = df_total["new"].fillna(0)
df_total.shape

(618, 76)

# Increase validation dataset

In [35]:
date = "2020-10-05"
mask_train = (df_total["upload_date"] < date) & (df_total["new"] == 0)
mask_val = (df_total["upload_date"] >= date) 

Xtrain, Xval = df_total[mask_train].drop("y", axis=1), df_total[mask_val].drop("y", axis=1)
ytrain, yval = df_total[mask_train].pop("y"), df_total[mask_val].pop("y")
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((263, 75), (300, 75), (263,), (300,))

### Title Modeling

In [39]:
title_train = Xtrain["title"]
title_val = Xval["title"]

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain[features], title_bow_train])
Xval_wtitle = hstack([Xval[features], title_bow_val])

Xtrain_wtitle.shape, Xval_wtitle.shape

((263, 225), (300, 225))

### Function for Validation

In [37]:
def val_score(Xtrain, ytrain, Xval, yval):
    
    mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
    mdl.fit(Xtrain, ytrain)
    p = mdl.predict_proba(Xval)[:, 1]
    return average_precision_score(yval, p), roc_auc_score(yval, p) 

In [40]:
val_score(Xtrain_wtitle, ytrain, Xval_wtitle, yval)

(0.224631005125001, 0.5309599999999999)

# Increase train dataset

In [51]:
date = "2020-10-05"
mask_train = (df_total["upload_date"] < date)
mask_val = (df_total["upload_date"] >= date) & (df_total["new"] == 0) 

Xtrain, Xval = df_total[mask_train].drop("y", axis=1), df_total[mask_val].drop("y", axis=1)
ytrain, yval = df_total[mask_train].pop("y"), df_total[mask_val].pop("y")
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((318, 75), (255, 75), (318,), (255,))

### Title modeling

In [52]:
title_train = Xtrain["title"]
title_val = Xval["title"]

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain[features], title_bow_train])
Xval_wtitle = hstack([Xval[features], title_bow_val])

Xtrain_wtitle.shape, Xval_wtitle.shape

((318, 268), (255, 268))

In [53]:
val_score(Xtrain_wtitle, ytrain, Xval_wtitle, yval)

(0.30226503556733264, 0.5944556226246367)

# Results

                             precision            roc_auc
Without active learning -> 0.2314598372172052, 0.5469483568075116

Increase validation -> 0.224631005125001, 0.5309599999999999

Increase training -> 0.30226503556733264, 0.5944556226246367

In [28]:
df_total.to_csv("csv\\data_ready.csv", index=False)
df_total.head()

,id,title,formats,thumbnails,description,upload_date,uploader,uploader_id,uploader_url,channel_id,...,protocol,preference,http_headers,release_date,release_year,days_since_upload,thumbnails_links,y,views_per_day,new
0,JyGGMyR3x5I,Machine Learning: Tutorial prático usando apen...,"[{'asr': 48000, 'filesize': 16323674, 'format_...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",✅ALURA COM 10% DE DESCONTO: https://www.alura....,2019-11-08,Filipe Deschamps,UCU5JicSrEM5A63jkJ2QvGYw,http://www.youtube.com/channel/UCU5JicSrEM5A63...,UCU5JicSrEM5A63jkJ2QvGYw,...,NaN,NaN,NaN,NaN,NaN,825.0,https://i.ytimg.com/vi_webp/JyGGMyR3x5I/maxres...,0.0,219.296970,0.0
1,0PrOA2JK6GQ,Machine Learning Explicado,"[{'asr': 48000, 'filesize': 5203175, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Machine learning (ou aprendizado de máquina) é...,2021-02-25,Ciência Todo Dia,CienciaTodoDia,http://www.youtube.com/user/CienciaTodoDia,UCn9Erjy00mpnWeLnRqhsA1g,...,NaN,NaN,NaN,NaN,NaN,350.0,https://i.ytimg.com/vi/0PrOA2JK6GQ/maxresdefau...,0.0,556.602857,0.0
2,t5z5lyrb-7s,Introdução ao Machine Learning (ML de Zero a 1...,"[{'asr': 48000, 'filesize': 2603732, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",This video is also available in English: https...,2019-11-13,TensorFlow,UC0rqucBdTuFTjJiefW5t-IQ,http://www.youtube.com/channel/UC0rqucBdTuFTjJ...,UC0rqucBdTuFTjJiefW5t-IQ,...,NaN,NaN,NaN,NaN,NaN,820.0,https://i.ytimg.com/vi_webp/t5z5lyrb-7s/maxres...,0.0,17.907317,0.0
3,5q87K1WaoFI,Computer Scientist Explains Machine Learning i...,"[{'asr': 48000, 'filesize': 9863039, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",WIRED has challenged computer scientist and Hi...,2021-08-18,WIRED,wired,http://www.youtube.com/user/wired,UCftwRNsjfRo08xYE31tkiyw,...,NaN,NaN,NaN,NaN,NaN,176.0,https://i.ytimg.com/vi/5q87K1WaoFI/maxresdefau...,0.0,5726.215909,0.0
4,qDmojpkd8rE,O que é Machine Learning (Aprendizado de Máqui...,"[{'format_id': '139', 'manifest_url': 'https:/...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Nesse vídeo vamos aprender o que é machine lea...,2019-06-21,Didática Tech,UC0BiVs5EYh57gzGVvhddjsA,http://www.youtube.com/channel/UC0BiVs5EYh57gz...,UC0BiVs5EYh57gzGVvhddjsA,...,NaN,NaN,NaN,NaN,NaN,965.0,https://i.ytimg.com/vi_webp/qDmojpkd8rE/maxres...,0.0,28.239378,0.0


In [5]:
def mapper_y_new(x):
    y = df_total.loc[df_total["thumbnails_links"] == x, "id"].tolist()
    if len(y) != 0:
        return True
    return False
df_unlabeled["annotated"] = df_unlabeled["thumbnails_links"].apply(lambda x: mapper_y_new(x))

In [6]:
mask = (df_unlabeled["annotated"] == False)

df_unlabeled[mask].sample(400, random_state=42)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [5]:
df_unlabeled = pd.read_csv("csv\\complete_labeled.csv")
df_unlabeled.head()

,Unnamed: 0,id,title,formats,thumbnails,description,upload_date,uploader,uploader_id,uploader_url,...,http_headers,release_date,release_year,days_since_upload,thumbnails_links,y,views_per_day,p,annotated,id_
0,230,-G5-kHF57OU,Apa itu Machine Learning? Inilah cara mudah me...,"[{'format_id': '139', 'manifest_url': 'https:/...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Machine learning pada dasarnya adalah proses y...,2019-03-17,The W Channel,d3tiuksw,http://www.youtube.com/user/d3tiuksw,...,NaN,NaN,NaN,1061.0,https://i.ytimg.com/vi/-G5-kHF57OU/hqdefault.j...,NaN,21.480679,0.094,False,-G5-kHF57OU
1,403,6dEUTmoXz0w,Machine Learning With R | Machine Learning Ful...,"[{'format_id': '139', 'manifest_url': 'https:/...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...","In this video on Machine Learning with R, you ...",2020-11-14,Simplilearn,Simplilearn,http://www.youtube.com/user/Simplilearn,...,NaN,NaN,NaN,453.0,https://i.ytimg.com/vi_webp/6dEUTmoXz0w/maxres...,NaN,74.997792,0.177,False,6dEUTmoXz0w
2,460,4wneZDEB3VA,Predictive Maintenance & Monitoring using Mach...,"[{'asr': 48000, 'filesize': 17706117, 'format_...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Learn how to build advanced predictive mainten...,2018-07-24,Google Workspace,GoogleApps,http://www.youtube.com/user/GoogleApps,...,NaN,NaN,NaN,1297.0,https://i.ytimg.com/vi_webp/4wneZDEB3VA/maxres...,NaN,32.144950,0.110,False,4wneZDEB3VA
3,461,-g_hyDI4lK4,#03 -What is Machine Learning and Its Applicat...,"[{'asr': 48000, 'filesize': 3737826, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...","At Skillari, We believe that Learning is not L...",2022-01-20,Skillari Machine Learning and AI,UCWlM9hB2TkXuUKIRbH1Y1Ug,http://www.youtube.com/channel/UCWlM9hB2TkXuUK...,...,NaN,NaN,NaN,21.0,https://i.ytimg.com/vi_webp/-g_hyDI4lK4/maxres...,NaN,676.285714,0.138,False,-g_hyDI4lK4
4,462,IzcX9bTJLj8,7 Steps to Build a Machine Learning Model,"[{'asr': 48000, 'filesize': 1401980, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Ease data preparation process with organizatio...,2021-04-12,Eye on Tech,TechnologyGuide,http://www.youtube.com/user/TechnologyGuide,...,NaN,NaN,NaN,304.0,https://i.ytimg.com/vi/IzcX9bTJLj8/maxresdefau...,NaN,9.263158,0.117,False,IzcX9bTJLj8


In [7]:
df_unlabeled["id_"] = df_unlabeled["id"]
df_unlabeled[['id_', 'title', 'thumbnails_links', 'view_count', 'uploader', 'description']].to_csv("csv\\complete_labeled_to_label.csv", index=False)

In [2]:
df2 = pd.read_csv("csv\\complete_labeled_to_label.csv")
df2.head()

,id_,title,y,thumbnails_links,view_count,uploader,description
0,#NOME?,Apa itu Machine Learning? Inilah cara mudah me...,0.0,https://i.ytimg.com/vi/-G5-kHF57OU/hqdefault.j...,22791,The W Channel,Machine learning pada dasarnya adalah proses y...
1,6dEUTmoXz0w,Machine Learning With R | Machine Learning Ful...,0.0,https://i.ytimg.com/vi_webp/6dEUTmoXz0w/maxres...,33974,Simplilearn,"In this video on Machine Learning with R, you ..."
2,4wneZDEB3VA,Predictive Maintenance & Monitoring using Mach...,1.0,https://i.ytimg.com/vi_webp/4wneZDEB3VA/maxres...,41692,Google Workspace,Learn how to build advanced predictive mainten...
3,#NOME?,#03 -What is Machine Learning and Its Applicat...,0.0,https://i.ytimg.com/vi_webp/-g_hyDI4lK4/maxres...,14202,Skillari Machine Learning and AI,"At Skillari, We believe that Learning is not L..."
4,IzcX9bTJLj8,7 Steps to Build a Machine Learning Model,1.0,https://i.ytimg.com/vi/IzcX9bTJLj8/maxresdefau...,2816,Eye on Tech,Ease data preparation process with organizatio...


In [3]:
df2[df2.y.notnull()].shape

(501, 7)

In [4]:
df = pd.read_csv("csv\\raw_data_labeled_ready.csv")
df.head()

,id,title,formats,thumbnails,description,upload_date,uploader,uploader_id,uploader_url,channel_id,...,tbr,protocol,preference,http_headers,release_date,release_year,days_since_upload,thumbnails_links,y,views_per_day
0,JyGGMyR3x5I,Machine Learning: Tutorial prático usando apen...,"[{'asr': 48000, 'filesize': 16323674, 'format_...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",✅ALURA COM 10% DE DESCONTO: https://www.alura....,2019-11-08,Filipe Deschamps,UCU5JicSrEM5A63jkJ2QvGYw,http://www.youtube.com/channel/UCU5JicSrEM5A63...,UCU5JicSrEM5A63jkJ2QvGYw,...,NaN,NaN,NaN,NaN,NaN,NaN,825.0,https://i.ytimg.com/vi_webp/JyGGMyR3x5I/maxres...,0.0,219.296970
1,0PrOA2JK6GQ,Machine Learning Explicado,"[{'asr': 48000, 'filesize': 5203175, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Machine learning (ou aprendizado de máquina) é...,2021-02-25,Ciência Todo Dia,CienciaTodoDia,http://www.youtube.com/user/CienciaTodoDia,UCn9Erjy00mpnWeLnRqhsA1g,...,NaN,NaN,NaN,NaN,NaN,NaN,350.0,https://i.ytimg.com/vi/0PrOA2JK6GQ/maxresdefau...,0.0,556.602857
2,t5z5lyrb-7s,Introdução ao Machine Learning (ML de Zero a 1...,"[{'asr': 48000, 'filesize': 2603732, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",This video is also available in English: https...,2019-11-13,TensorFlow,UC0rqucBdTuFTjJiefW5t-IQ,http://www.youtube.com/channel/UC0rqucBdTuFTjJ...,UC0rqucBdTuFTjJiefW5t-IQ,...,NaN,NaN,NaN,NaN,NaN,NaN,820.0,https://i.ytimg.com/vi_webp/t5z5lyrb-7s/maxres...,0.0,17.907317
3,5q87K1WaoFI,Computer Scientist Explains Machine Learning i...,"[{'asr': 48000, 'filesize': 9863039, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",WIRED has challenged computer scientist and Hi...,2021-08-18,WIRED,wired,http://www.youtube.com/user/wired,UCftwRNsjfRo08xYE31tkiyw,...,NaN,NaN,NaN,NaN,NaN,NaN,176.0,https://i.ytimg.com/vi/5q87K1WaoFI/maxresdefau...,0.0,5726.215909
4,qDmojpkd8rE,O que é Machine Learning (Aprendizado de Máqui...,"[{'format_id': '139', 'manifest_url': 'https:/...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Nesse vídeo vamos aprender o que é machine lea...,2019-06-21,Didática Tech,UC0BiVs5EYh57gzGVvhddjsA,http://www.youtube.com/channel/UC0BiVs5EYh57gz...,UC0BiVs5EYh57gzGVvhddjsA,...,NaN,NaN,NaN,NaN,NaN,NaN,965.0,https://i.ytimg.com/vi_webp/qDmojpkd8rE/maxres...,0.0,28.239378


In [5]:
def mapper_y(x):
    y = df2.loc[df2["thumbnails_links"] == x, "y"].tolist()
    if len(y) != 0:
        return y[0]
    return df.loc[df["thumbnails_links"] == x, "y"].tolist()[0]

df["y"] = df["thumbnails_links"].apply(lambda x: mapper_y(x))

In [6]:
df_total = pd.read_csv("csv\\data_ready.csv")
df_total.head()

,id,title,formats,thumbnails,description,upload_date,uploader,uploader_id,uploader_url,channel_id,...,protocol,preference,http_headers,release_date,release_year,days_since_upload,thumbnails_links,y,views_per_day,new
0,JyGGMyR3x5I,Machine Learning: Tutorial prático usando apen...,"[{'asr': 48000, 'filesize': 16323674, 'format_...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",✅ALURA COM 10% DE DESCONTO: https://www.alura....,2019-11-08,Filipe Deschamps,UCU5JicSrEM5A63jkJ2QvGYw,http://www.youtube.com/channel/UCU5JicSrEM5A63...,UCU5JicSrEM5A63jkJ2QvGYw,...,NaN,NaN,NaN,NaN,NaN,825.0,https://i.ytimg.com/vi_webp/JyGGMyR3x5I/maxres...,0.0,219.296970,0.0
1,0PrOA2JK6GQ,Machine Learning Explicado,"[{'asr': 48000, 'filesize': 5203175, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Machine learning (ou aprendizado de máquina) é...,2021-02-25,Ciência Todo Dia,CienciaTodoDia,http://www.youtube.com/user/CienciaTodoDia,UCn9Erjy00mpnWeLnRqhsA1g,...,NaN,NaN,NaN,NaN,NaN,350.0,https://i.ytimg.com/vi/0PrOA2JK6GQ/maxresdefau...,0.0,556.602857,0.0
2,t5z5lyrb-7s,Introdução ao Machine Learning (ML de Zero a 1...,"[{'asr': 48000, 'filesize': 2603732, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",This video is also available in English: https...,2019-11-13,TensorFlow,UC0rqucBdTuFTjJiefW5t-IQ,http://www.youtube.com/channel/UC0rqucBdTuFTjJ...,UC0rqucBdTuFTjJiefW5t-IQ,...,NaN,NaN,NaN,NaN,NaN,820.0,https://i.ytimg.com/vi_webp/t5z5lyrb-7s/maxres...,0.0,17.907317,0.0
3,5q87K1WaoFI,Computer Scientist Explains Machine Learning i...,"[{'asr': 48000, 'filesize': 9863039, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",WIRED has challenged computer scientist and Hi...,2021-08-18,WIRED,wired,http://www.youtube.com/user/wired,UCftwRNsjfRo08xYE31tkiyw,...,NaN,NaN,NaN,NaN,NaN,176.0,https://i.ytimg.com/vi/5q87K1WaoFI/maxresdefau...,0.0,5726.215909,0.0
4,qDmojpkd8rE,O que é Machine Learning (Aprendizado de Máqui...,"[{'format_id': '139', 'manifest_url': 'https:/...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Nesse vídeo vamos aprender o que é machine lea...,2019-06-21,Didática Tech,UC0BiVs5EYh57gzGVvhddjsA,http://www.youtube.com/channel/UC0BiVs5EYh57gz...,UC0BiVs5EYh57gzGVvhddjsA,...,NaN,NaN,NaN,NaN,NaN,965.0,https://i.ytimg.com/vi_webp/qDmojpkd8rE/maxres...,0.0,28.239378,0.0


In [7]:
def mapper_y(x):
    y = df_total.loc[df_total["thumbnails_links"] == x, "y"].tolist()
    if len(y) != 0:
        return y[0]
    return df.loc[df["thumbnails_links"] == x, "y"].tolist()[0]

df["y"] = df["thumbnails_links"].apply(lambda x: mapper_y(x))

In [8]:
df.head()

,id,title,formats,thumbnails,description,upload_date,uploader,uploader_id,uploader_url,channel_id,...,tbr,protocol,preference,http_headers,release_date,release_year,days_since_upload,thumbnails_links,y,views_per_day
0,JyGGMyR3x5I,Machine Learning: Tutorial prático usando apen...,"[{'asr': 48000, 'filesize': 16323674, 'format_...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",✅ALURA COM 10% DE DESCONTO: https://www.alura....,2019-11-08,Filipe Deschamps,UCU5JicSrEM5A63jkJ2QvGYw,http://www.youtube.com/channel/UCU5JicSrEM5A63...,UCU5JicSrEM5A63jkJ2QvGYw,...,NaN,NaN,NaN,NaN,NaN,NaN,825.0,https://i.ytimg.com/vi_webp/JyGGMyR3x5I/maxres...,0.0,219.296970
1,0PrOA2JK6GQ,Machine Learning Explicado,"[{'asr': 48000, 'filesize': 5203175, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Machine learning (ou aprendizado de máquina) é...,2021-02-25,Ciência Todo Dia,CienciaTodoDia,http://www.youtube.com/user/CienciaTodoDia,UCn9Erjy00mpnWeLnRqhsA1g,...,NaN,NaN,NaN,NaN,NaN,NaN,350.0,https://i.ytimg.com/vi/0PrOA2JK6GQ/maxresdefau...,0.0,556.602857
2,t5z5lyrb-7s,Introdução ao Machine Learning (ML de Zero a 1...,"[{'asr': 48000, 'filesize': 2603732, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",This video is also available in English: https...,2019-11-13,TensorFlow,UC0rqucBdTuFTjJiefW5t-IQ,http://www.youtube.com/channel/UC0rqucBdTuFTjJ...,UC0rqucBdTuFTjJiefW5t-IQ,...,NaN,NaN,NaN,NaN,NaN,NaN,820.0,https://i.ytimg.com/vi_webp/t5z5lyrb-7s/maxres...,0.0,17.907317
3,5q87K1WaoFI,Computer Scientist Explains Machine Learning i...,"[{'asr': 48000, 'filesize': 9863039, 'format_i...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",WIRED has challenged computer scientist and Hi...,2021-08-18,WIRED,wired,http://www.youtube.com/user/wired,UCftwRNsjfRo08xYE31tkiyw,...,NaN,NaN,NaN,NaN,NaN,NaN,176.0,https://i.ytimg.com/vi/5q87K1WaoFI/maxresdefau...,0.0,5726.215909
4,qDmojpkd8rE,O que é Machine Learning (Aprendizado de Máqui...,"[{'format_id': '139', 'manifest_url': 'https:/...","[{'height': 94, 'url': 'https://i.ytimg.com/vi...",Nesse vídeo vamos aprender o que é machine lea...,2019-06-21,Didática Tech,UC0BiVs5EYh57gzGVvhddjsA,http://www.youtube.com/channel/UC0BiVs5EYh57gz...,UC0BiVs5EYh57gzGVvhddjsA,...,NaN,NaN,NaN,NaN,NaN,NaN,965.0,https://i.ytimg.com/vi_webp/qDmojpkd8rE/maxres...,0.0,28.239378


In [9]:
df.to_csv("csv\\total_data.csv", index=False)